# Classical Spin Hamiltonians

In [64]:
import sys
sys.path.append("../..")
import QuantumSparse as qs
import numpy as np
import sympy
from sympy import symbols,cos,sin,Matrix, hessian,lambdify
from scipy.optimize import minimize

In [65]:
S     = 3./2.
NSpin = 8
SpinValues = np.full(NSpin,S)
print(SpinValues)

[1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5]


### angular variables

In [66]:
# [0,pi]
theta = [sympy.symbols('\\theta_%d' % i) for i in range(1,NSpin+1)] #,real=True,positive=True
# [0,2pi)
phi   = [sympy.symbols('\\phi_%d'   % i) for i in range(1,NSpin+1)] #,real=True,positive=True
print(theta)
print(phi)

[\theta_1, \theta_2, \theta_3, \theta_4, \theta_5, \theta_6, \theta_7, \theta_8]
[\phi_1, \phi_2, \phi_3, \phi_4, \phi_5, \phi_6, \phi_7, \phi_8]


### spin variables

In [67]:
Sx = [sympy.symbols('S^x_%d' % i) for i in range(1,NSpin+1)]
Sy = [sympy.symbols('S^y_%d' % i) for i in range(1,NSpin+1)]
Sz = [sympy.symbols('S^z_%d' % i) for i in range(1,NSpin+1)]
print(Sx)
print(Sy)
print(Sz)

[S^x_1, S^x_2, S^x_3, S^x_4, S^x_5, S^x_6, S^x_7, S^x_8]
[S^y_1, S^y_2, S^y_3, S^y_4, S^y_5, S^y_6, S^y_7, S^y_8]
[S^z_1, S^z_2, S^z_3, S^z_4, S^z_5, S^z_6, S^z_7, S^z_8]


### Spherical coordinates

In [5]:
def spherical_coordinates(r,theta,phi):
    x = r*cos(phi)*sin(theta)
    y = r*sin(phi)*sin(theta)
    z = r*cos(theta)
    return x,y,z

In [6]:
for i in range(NSpin):
    Sx[i],Sy[i],Sz[i] = spherical_coordinates(S,theta[i],phi[i])
    
print(Sx)
print(Sy)
print(Sz)

[1.5*sin(\theta_1)*cos(\phi_1), 1.5*sin(\theta_2)*cos(\phi_2), 1.5*sin(\theta_3)*cos(\phi_3), 1.5*sin(\theta_4)*cos(\phi_4), 1.5*sin(\theta_5)*cos(\phi_5), 1.5*sin(\theta_6)*cos(\phi_6), 1.5*sin(\theta_7)*cos(\phi_7), 1.5*sin(\theta_8)*cos(\phi_8)]
[1.5*sin(\phi_1)*sin(\theta_1), 1.5*sin(\phi_2)*sin(\theta_2), 1.5*sin(\phi_3)*sin(\theta_3), 1.5*sin(\phi_4)*sin(\theta_4), 1.5*sin(\phi_5)*sin(\theta_5), 1.5*sin(\phi_6)*sin(\theta_6), 1.5*sin(\phi_7)*sin(\theta_7), 1.5*sin(\phi_8)*sin(\theta_8)]
[1.5*cos(\theta_1), 1.5*cos(\theta_2), 1.5*cos(\theta_3), 1.5*cos(\theta_4), 1.5*cos(\theta_5), 1.5*cos(\theta_6), 1.5*cos(\theta_7), 1.5*cos(\theta_8)]


### Hamiltonian

In [7]:
H = 0
Jx,Jy,Jz = symbols('J_x J_y J_z')
D,E      = symbols('D E')
dx,dy,dz = symbols('d_x d_y d_z')

In [8]:
def Hamiltonian(Sx,Sy,Sz,Jx,Jy,Jz,D,E,dx,dy,dz):
    H = 0
    H += qs.Heisenberg(Sx,Sy,Sz,couplings=[Jx,Jy,Jz],nn=1,opts={"sympy":True}).simplify()
    H += qs.anisotropy(Sz,couplings=D,opts={"sympy":True}).simplify()
    H += qs.rombicity(Sx,Sy,couplings=E,opts={"sympy":True}).simplify()
    H += qs.DM(Sx,Sy,Sz,couplings=[dx,dy,dz],opts={"sympy":True}).simplify()
    return H    

### gradient

In [10]:
# https://stackoverflow.com/questions/60164477/define-gradient-and-hessian-function-in-python/60165226#60165226
gradient = lambda f, v: Matrix([f]).jacobian(v)

### lambdify

Trasformo le costanti di accoppiamento in valori numerici e ricalcolo l'Hamiltoniana, il suo gradiente e l'hessiana.

Poi trasformo queste funzioni da espressioni sympy a funzioni python per poi minimizzarne il valore dell'Hamiltoniana.

In [11]:
Jx =  0.433
Jy =  0.437
Jz =  0.629
D  = -0.585
E  = -0.005
dx =  0.40143
dy = -0.40078
dz =  0.82405
H     = Hamiltonian(Sx,Sy,Sz,Jx,Jy,Jz,D,E,dx,dy,dz)
grad_ = gradient(H, theta+phi)
hess_ = hessian(H, theta+phi)

In [12]:
energy = lambdify([theta+phi],H,'numpy')
grad   = lambdify([theta+phi],grad_,'numpy')
hess   = lambdify([theta+phi],hess_,'numpy')

### Minimization

Documentation of `scipy.optimize.minimize` can be found here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

In [13]:
bounds = [(0,np.pi) for i in range(NSpin)] + [(0,2*np.pi) for i in range(NSpin)]

In [52]:
N = 10
E = np.zeros(N)
angles = np.zeros((N,NSpin*2))
for n in range(N):
    print(n+1,"/",N)
    x0 = np.random.rand(NSpin*2)*np.asarray( [ np.pi for i in range(NSpin)] + [ 2*np.pi for i in range(NSpin)] )
    res = minimize(fun=energy,\
                   x0=x0,\
                   jac=lambda x : grad(x).T.flatten(),\
                   hess=hess,\
                   bounds=bounds,\
                   method='L-BFGS-B', tol=1e-4)
    #print(res)
    results = res.x*180/np.pi
    
    E[n] = res.fun
    angles[n,:] = res.x    
    print("energy: ",res.fun)
    print(" theta: ",results[0:NSpin])
    print("   phi: ",results[NSpin:])
    print()

1 / 10
energy:  -21.246596307892283
 theta:  [ 16.13494348 133.5860519  118.9321616   45.12898511  58.35157913
 180.           2.41513666 173.72145242]
   phi:  [130.49628006 360.         248.11917464 165.5659775   46.03053727
  77.46673389 359.88035979 235.36922355]

2 / 10
energy:  -21.85197645624395
 theta:  [5.24727649e-03 1.80000000e+02 0.00000000e+00 1.80000000e+02
 0.00000000e+00 1.80000000e+02 1.07379074e-01 1.79825140e+02]
   phi:  [293.01659825  73.27742748 299.82610218 189.4580747  183.85251127
  52.78431221 175.57904795  80.38557805]

3 / 10
energy:  -22.315605563885015
 theta:  [180.          10.73346376 145.05420046  88.15932847  39.77253795
 141.37720495  90.2301004   30.12501838]
   phi:  [104.51295434 140.85500247   8.76655028 227.16547289  95.48687664
 351.3780188  219.78694629  80.73980123]

4 / 10
energy:  -22.234684065458577
 theta:  [159.20819807   0.         180.          37.8689819   73.58976206
 145.91382881  51.30593299  61.59721789]
   phi:  [279.54890157  28

In [53]:
print(" index:",np.argmin(E))
print("energy:",np.min(E))

 index: 5
energy: -22.657962993197415


In [54]:
matrix = hess(angles[np.argmin(E)])
np.linalg.eig(matrix)[0]

array([8.21832068, 7.71430034, 6.61660887, 5.15865261, 5.12036936,
       4.65254715, 0.10115893, 0.24112748, 0.68518128, 1.25398354,
       1.61832668, 1.42407295, 3.68513108, 2.73577833, 2.85447229,
       2.88563001])

In [56]:
H = qs.Heisenberg(Sx,Sy,Sz,couplings=1,nn=1,opts={"sympy":True}).simplify()
grad_ = gradient(H, theta+phi)
hess_ = hessian(H, theta+phi)
energy = lambdify([theta+phi],H,'numpy')
grad   = lambdify([theta+phi],grad_,'numpy')
hess   = lambdify([theta+phi],hess_,'numpy')
bounds = [(0,np.pi) for i in range(NSpin)] + [(0,2*np.pi) for i in range(NSpin)]

In [58]:
N = 10
E = np.zeros(N)
angles = np.zeros((N,NSpin*2))
for n in range(N):
    print(n+1,"/",N)
    x0 = np.random.rand(NSpin*2)*np.asarray( [ np.pi for i in range(NSpin)] + [ 2*np.pi for i in range(NSpin)] )
    res = minimize(fun=energy,\
                   x0=x0,\
                   jac=lambda x : grad(x).T.flatten(),\
                   hess=hess,\
                   bounds=bounds,\
                   method='L-BFGS-B', tol=1e-4)
    #print(res)
    results = res.x*180/np.pi
    
    E[n] = res.fun
    angles[n,:] = res.x    
    print("energy: ",res.fun)
    print(" theta: ",results[0:NSpin])
    print("   phi: ",results[NSpin:])
    print()

1 / 10
energy:  -17.999757813556755
 theta:  [180.           0.         179.35099095   0.49955126 180.
   0.         180.           0.        ]
   phi:  [1.35889904e-01 1.97734795e+02 3.43938898e+02 1.75701659e+02
 1.71697714e+02 8.70604995e+01 3.60000000e+02 1.68967511e+02]

2 / 10
energy:  -17.99950597235303
 theta:  [ 45.13833812 134.38022282  45.70772302 134.16658807  45.79660125
 134.4706937   45.32790121 135.05417105]
   phi:  [  0.         180.02177849 359.3923961  178.53875953 358.77729549
 178.9006755  358.96616544 179.55694046]

3 / 10
energy:  -17.999391904330366
 theta:  [  8.67349588 171.42955729   8.53940524 171.3292478    8.74795605
 171.39566523   8.63905726 171.23778998]
   phi:  [  4.65470296 188.42563755   8.17389145 181.59867784   0.
 178.51829408 360.         182.18743552]

4 / 10
energy:  -18.0
 theta:  [  0. 180.   0. 180.   0. 180.   0. 180.]
   phi:  [161.96771752 318.46303865 104.53838461 296.70069898  91.37280582
 126.63600153   6.84518432   0.        ]

5 / 

In [62]:
matrix = hess(angles[np.argmin(E)])
np.round(np.linalg.eig(matrix)[0],6)

array([ 8.454282,  7.574851,  6.889659,  4.82849 ,  4.17151 ,  2.110341,
        0.545718,  1.425149, -0.      , -0.      , -0.      , -0.      ,
       -0.      ,  0.      ,  0.      ,  0.      ])

In [63]:
grad(angles[np.argmin(E)])

array([[-5.14694185e-16, -5.51091060e-16, -4.98000846e-16,
        -5.51091060e-16, -2.40732504e-17, -5.51091060e-16,
         1.36680763e-16, -5.51091060e-16,  0.00000000e+00,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])